# Capstone Project 
# *Find myself the best neighborhood to live in Barcelona*

### Yu Deng

Before I start, it's neccessary to import all the packages required.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim

from pandas.io.json import json_normalize

import matplotlib as plt

from sklearn.cluster import KMeans

import folium

print('Succeed!')

Succeed!


### 1. Use geopy library to get the geographical coordinate of ESADE Business School

In [3]:
esade_add = 'ESADE Business School'

geolocator = Nominatim(user_agent="my-application", timeout=10)
location = geolocator.geocode(esade_add)
esade_lat = location.latitude
esade_lng = location.longitude
print('The geographical coordinate of ESADE Business School are {}, {}.'.format(esade_lat, esade_lng))

The geographical coordinate of ESADE Business School are 41.3947771, 2.1150794468981124.


### 2. Choose the districts around ESADE and find out their subordinated neighborhoods

On the wikipedia page ESADE Business School https://en.wikipedia.org/wiki/Districts_of_Barcelona, we can find the basic informations about every districts in Barcelona, including their sizes, populations, neighborhoods and so on. Firstly I scrape the table and transform it into a dataframe.

In [4]:
url = "http://en.wikipedia.org/wiki/Districts_of_Barcelona"
barce_wiki = pd.read_html(url)[3]
barce_wiki.head()

,Number,District,Size km2,Population,Density inhabitants/km2,Neighbourhoods,Councilman[2],Party
0,1,Ciutat Vella,4.49,111290,24786,"La Barceloneta, El Gòtic, El Raval, Sant Pere,...",Jordi Rabassa i Massons,Barcelona en Comú
1,2,Eixample,7.46,262485,35586,"L'Antiga Esquerra de l'Eixample, La Nova Esque...",Jordi Martí Grau,Barcelona en Comú
2,3,Sants-Montjuïc,21.35,177636,8321,"La Bordeta, la Font de la Guatlla, Hostafrancs...",Marc Serra Solé,Barcelona en Comú
3,4,Les Corts,6.08,82588,13584,"les Corts, la Maternitat i Sant Ramon, Pedralbes",Xavier Marcé Carol,Socialists' Party of Catalonia
4,5,Sarrià-Sant Gervasi,20.09,140461,6992,"El Putget i Farró, Sarrià, Sant Gervasi - la B...",Albert Batlle i Bastardas,Socialists' Party of Catalonia


I do some basic cleanings and only keep *District*, *Density* and *Neighborhoods* columns.

In [5]:
barce_district = barce_wiki.drop(['Number', 'Size km2', 'Population', 'Councilman[2]', 'Party'], axis=1)
barce_district.rename(columns = {'Density inhabitants/km2': 'Density'}, inplace=True)
barce_district.head()

,District,Density,Neighbourhoods
0,Ciutat Vella,24786,"La Barceloneta, El Gòtic, El Raval, Sant Pere,..."
1,Eixample,35586,"L'Antiga Esquerra de l'Eixample, La Nova Esque..."
2,Sants-Montjuïc,8321,"La Bordeta, la Font de la Guatlla, Hostafrancs..."
3,Les Corts,13584,"les Corts, la Maternitat i Sant Ramon, Pedralbes"
4,Sarrià-Sant Gervasi,6992,"El Putget i Farró, Sarrià, Sant Gervasi - la B..."


Use the same geopy package to get the geographical coordinates of each district.

In [6]:
districts = barce_district.iloc[:,0]
districts_lat = []
districts_lng = []

for district in districts:
    geolocator = Nominatim(user_agent="my-application")
    location = geolocator.geocode(district + ", Barcelona", timeout=10)
    districts_lat.append(location.latitude)
    districts_lng.append(location.longitude)

barce_district['Latitude'] = pd.Series(districts_lat)
barce_district['Longitude'] = pd.Series(districts_lng)

barce_district

,District,Density,Neighbourhoods,Latitude,Longitude
0,Ciutat Vella,24786,"La Barceloneta, El Gòtic, El Raval, Sant Pere,...",41.374962,2.173265
1,Eixample,35586,"L'Antiga Esquerra de l'Eixample, La Nova Esque...",41.393394,2.166085
2,Sants-Montjuïc,8321,"La Bordeta, la Font de la Guatlla, Hostafrancs...",41.340234,2.133347
3,Les Corts,13584,"les Corts, la Maternitat i Sant Ramon, Pedralbes",41.385244,2.132863
4,Sarrià-Sant Gervasi,6992,"El Putget i Farró, Sarrià, Sant Gervasi - la B...",41.413043,2.108356
5,Gràcia,28660,"Vila de Gràcia, el Camp d'en Grassot i Gràcia ...",41.410171,2.155136
6,Horta-Guinardó,14217,"El Baix Guinardó, El Guinardó, Can Baró, El Ca...",41.428540,2.143597
7,Nou Barris,20520,"Can Peguera, Canyelles, Ciutat Meridiana, La G...",41.446727,2.172565
8,Sant Andreu,21737,"Baró de Viver, Bon Pastor, El Congrés i els In...",41.437439,2.196859
9,Sant Martí,20466,"El Besòs i el Maresme, el Clot, El Camp de l'A...",41.406782,2.203655


Use the folium package, I can plot coordinates of ESADE Business School and all districts on the map.

In [7]:
barce_add = 'Barcelona'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(barce_add, timeout=10)
barce_lat = location.latitude
barce_lng = location.longitude

barce_map = folium.Map(location=[barce_lat, barce_lng], zoom_start=12)

for lat, lng, district in zip(barce_district['Latitude'], barce_district['Longitude'], barce_district['District']):
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=district,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3,
        parse_html=False).add_to(barce_map)  

folium.Marker([esade_lat, esade_lng], popup='ESADE').add_to(barce_map)

barce_map

Obviously I want to live near my school since it is easier to commute. So I choose the top 5 closest districts displayed by the map above, which are ***Les Corts, Sarrià-Sant Gervasi, Eixample, Gràcia and Horta-Guinardó***.

In [8]:
barce_district_candi = barce_district.iloc[[1,3,4,5,6],:].reset_index(drop = True)
barce_district_candi

,District,Density,Neighbourhoods,Latitude,Longitude
0,Eixample,35586,"L'Antiga Esquerra de l'Eixample, La Nova Esque...",41.393394,2.166085
1,Les Corts,13584,"les Corts, la Maternitat i Sant Ramon, Pedralbes",41.385244,2.132863
2,Sarrià-Sant Gervasi,6992,"El Putget i Farró, Sarrià, Sant Gervasi - la B...",41.413043,2.108356
3,Gràcia,28660,"Vila de Gràcia, el Camp d'en Grassot i Gràcia ...",41.410171,2.155136
4,Horta-Guinardó,14217,"El Baix Guinardó, El Guinardó, Can Baró, El Ca...",41.428540,2.143597


### 3. Acquire the venues data in neighborhoods

In this section, I will split the dataframe of districts into neighborhoods within them. Than use Foursquare.com to explore venues in each neighborhood. Since the venues data naturally leads me towards next section: clustering and further analysis, I leave it in the third part of my report. Here I will simply finish the split work.

In [9]:
neigh_split = barce_district_candi['Neighbourhoods'].str.split(', ', expand=True).stack().to_frame() 
neigh_split = neigh_split.reset_index(level=1, drop=True).rename(columns={0:'Neighborhood'}) 

barce_neigh = barce_district_candi.join(neigh_split).loc[:,['District', 'Neighborhood']].reset_index(drop = True)
barce_neigh.head()

,District,Neighborhood
0,Eixample,L'Antiga Esquerra de l'Eixample
1,Eixample,La Nova Esquerra de l'Eixample
2,Eixample,Dreta de l'Eixample
3,Eixample,Fort Pienc
4,Eixample,Sagrada Família


Use the same method to acquire the geographical coordinate of each neighborhood.

In [14]:
neighs = barce_neigh.iloc[:,1]

neigh_lat = []
neigh_lng = []

for neigh in neighs:
    geolocator = Nominatim(user_agent="my-application")
    location = geolocator.geocode(neigh + ", Barcelona", timeout=10)
    neigh_lat.append(location.latitude)
    neigh_lng.append(location.longitude)

barce_neigh['Latitude'] = pd.Series(neigh_lat)
barce_neigh['Longitude'] = pd.Series(neigh_lng)

barce_neigh

,District,Neighborhood,Latitude,Longitude
0,Eixample,L'Antiga Esquerra de l'Eixample,41.388765,2.156597
1,Eixample,La Nova Esquerra de l'Eixample,41.382816,2.149966
2,Eixample,Dreta de l'Eixample,41.394124,2.166471
3,Eixample,Fort Pienc,41.395925,2.182325
4,Eixample,Sagrada Família,41.403479,2.174410
5,Eixample,Sant Antoni,41.380578,2.160102
6,Les Corts,les Corts,41.385244,2.132863
7,Les Corts,la Maternitat i Sant Ramon,41.381975,2.115474
8,Les Corts,Pedralbes,41.390140,2.112218
9,Sarrià-Sant Gervasi,El Putget i Farró,41.407476,2.143283


In [15]:
barce_neigh.to_csv('Barcelona_Neighborhoods.csv')

In [11]:
barce_neigh = pd.read_csv('Barcelona_Neighborhoods.csv', index_col=0)

,District,Neighborhood,Latitude,Longitude
0,Eixample,L'Antiga Esquerra de l'Eixample,41.388765,2.156597
1,Eixample,La Nova Esquerra de l'Eixample,41.382816,2.149966
2,Eixample,Dreta de l'Eixample,41.394124,2.166471
3,Eixample,Fort Pienc,41.395925,2.182325
4,Eixample,Sagrada Família,41.403479,2.174410


In this part I use Foursqaure to explore venues within each neighborhoods, figure out the characteristics and similarities of these neighborhoods and deploy machine learning to do clustering.

To begin with, define Foursquare credentials and version.

In [12]:
CLIENT_ID = '5IK3E2GVEJJOHW0KLB1OJLSMUDU45GS1H1ZOEFIGKWAWPOKG'
CLIENT_SECRET = '2NBAOZ2QHNIXEOA3BCWXXWQMIRTGAYCSGAW2QPFFUPQNCLII'
VERSION = '20200320'

Create a function to detect venues around specific neibourhoods within 1 kilometer. Set the upper limit as 100.

In [22]:
import requests

def getVenues(dists, neighs, lats, lngs, radius = 1000, LIMIT = 100):
    
    venues_list=[]
    for dist, neigh, lat, lng in zip(dists, neighs, lats, lngs):
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            dist,
            neigh, 
            v['venue']['name'],  
            v['venue']['categories'][0]['name']) for v in results])

    venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    venues.columns = ['District',
                  'Neighborhood', 
                  'Venue',  
                  'Venue Category']

    return(venues)

barce_venues = getVenues(dists = barce_neigh['District'],
                                   neighs = barce_neigh['Neighborhood'],
                                   lats = barce_neigh['Latitude'],
                                   lngs = barce_neigh['Longitude']
                                  )

barce_venues.head()

,District,Neighborhood,Venue,Venue Category
0,Eixample,L'Antiga Esquerra de l'Eixample,Roast Club Café,Coffee Shop
1,Eixample,L'Antiga Esquerra de l'Eixample,Foc i Oli,Sandwich Place
2,Eixample,L'Antiga Esquerra de l'Eixample,Tandem,Cocktail Bar
3,Eixample,L'Antiga Esquerra de l'Eixample,The Corner Hotel Barcelona,Hotel
4,Eixample,L'Antiga Esquerra de l'Eixample,Mikkeller Bar Barcelona,Beer Bar


After I successfully retrieved the data from Foursquare, I have to check the number of the venues. Futhurmore, and most importantly, since their is max requests restriction imposed by Foursquare, the data should be exported and saved in csv form.

In [26]:
barce_venues.to_csv('Barcelona_Venues.csv')

print('There are {} records of venues in the dataset.'.format(barce_venues.shape[0]))

There are 2513 records of venues in the dataset.


In [13]:
barce_venues = pd.read_csv('Barcelona_Venues.csv', index_col=0)

,District,Neighborhood,Venue,Venue Category
0,Eixample,L'Antiga Esquerra de l'Eixample,Roast Club Café,Coffee Shop
1,Eixample,L'Antiga Esquerra de l'Eixample,Foc i Oli,Sandwich Place
2,Eixample,L'Antiga Esquerra de l'Eixample,Tandem,Cocktail Bar
3,Eixample,L'Antiga Esquerra de l'Eixample,The Corner Hotel Barcelona,Hotel
4,Eixample,L'Antiga Esquerra de l'Eixample,Mikkeller Bar Barcelona,Beer Bar


Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [14]:
barce_venues_dummies = pd.get_dummies(barce_venues[['Venue Category']], prefix="", prefix_sep="")
barce_venues_dummies = pd.concat([barce_venues[['District', 'Neighborhood']], barce_venues_dummies], axis=1)

barce_venues_means = barce_venues_dummies.groupby('Neighborhood').mean().reset_index()
barce_venues_means = pd.merge(barce_venues_means, barce_neigh, how='left', on=['Neighborhood'])
barce_venues_means.head()

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Animal Shelter,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bistro,Board Shop,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Butcher,Cafeteria,Café,Cambodian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doctor's Office,Dog Run,Donut Shop,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Multiplex,Museum,Music Venue,National Park,Nightclub,Noodle House,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pharmacy,Photography Lab,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Restaurant,Road,Rock Club,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio,District,Latitude,Longitude
0,Can Baró,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.01,0.0,0.00,0.01,0.00,0.050000,0.070000,0.0,0.00,0.00,0.01,0.00,0.00,0.0,0.0,0.01,0.01,0.00,0.0,0.02,0.00,0.0,0.0,0.01000,0.0,0.0,0.04000,0.01,0.0,0.03000,0.00,0.0,0.0,0.000000,0.000000,0.01000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.000000,0.0,0.02,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.00,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.01,0.0,0.0,0.00,0.000000,0.01,0.0,0.0,0.04000,0.05000,0.00000,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.00000,0.000000,0.01,0.00,0.0,0.00,0.0,0.010000,0.0,0.00,0.0,0.0,0.0,0.020000,0.01,0.01,0.00,0.0,0.0,0.00,0.0,0.01,0.000000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.020000,0.000000

Use k-means clustering, I can categorize the candidates into 5 groups, based on the occurences of different venues within each neighborhood. 

In [15]:
kclusters = 5

barce_clustering = barce_venues_means.drop(['Neighborhood', 'District', 'Latitude', 'Longitude'], axis=1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(barce_clustering)

barce_venues_means.insert(1, 'Cluster', kmeans.labels_)

barce_venues_means.head()

,Neighborhood,Cluster,Accessories Store,African Restaurant,American Restaurant,Animal Shelter,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bistro,Board Shop,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Butcher,Cafeteria,Café,Cambodian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doctor's Office,Dog Run,Donut Shop,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Multiplex,Museum,Music Venue,National Park,Nightclub,Noodle House,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pharmacy,Photography Lab,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Restaurant,Road,Rock Club,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio,District,Latitude,Longitude
0,Can Baró,4,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.01,0.0,0.00,0.01,0.00,0.050000,0.070000,0.0,0.00,0.00,0.01,0.00,0.00,0.0,0.0,0.01,0.01,0.00,0.0,0.02,0.00,0.0,0.0,0.01000,0.0,0.0,0.04000,0.01,0.0,0.03000,0.00,0.0,0.0,0.000000,0.000000,0.01000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.000000,0.0,0.02,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.00,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.01,0.0,0.0,0.00,0.000000,0.01,0.0,0.0,0.04000,0.05000,0.00000,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.00000,0.000000,0.01,0.00,0.0,0.00,0.0,0.010000,0.0,0.00,0.0,0.0,0.0,0.020000,0.01,0.01,0.00,0.0,0.0,0.00,0.0,0.01,0.000000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.02000

Plot different clusters on the map.

In [47]:
clusters_map = folium.Map(location=[barce_venues_means.mean()['Latitude'], barce_venues_means.mean()['Longitude']], zoom_start=13)

colors = ['red', 'green', 'blue', 'purple', 'yellow']

markers_colors = []
for lat, lon, poi, cluster in zip(barce_venues_means['Latitude'], barce_venues_means['Longitude'], barce_venues_means['Neighborhood'], barce_venues_means['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster+1), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=15,
        popup=label,
        color=colors[cluster],
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.5).add_to(clusters_map)
    
folium.Marker([esade_lat, esade_lng], popup='ESADE').add_to(clusters_map)
       
clusters_map

We can find some interesing figures of these clusters. Cluster 4 (yellow) and cluster 2 (blue) are located really closely, and the other three clusters are not within a bigger circle as to out common sense, but disperse in three long parellel strips.

Futhermore, list the top 10 most common venues in each neighborhoods.

In [23]:
columns = ['Neighborhood', 'Cluster', 'Latitude', 'Longitude']

for index in np.arange(10):
    columns.append('No.{}'.format((index)+1))

barce_venues_sorted = pd.DataFrame(columns=columns)
barce_venues_sorted[columns[0:4]] = barce_venues_means[columns[0:4]]

for num in np.arange(barce_venues_means.shape[0]):
    categories = barce_venues_means.iloc[num, :].iloc[2:-3]
    categories_sorted = categories.sort_values(ascending=False)
    barce_venues_sorted.iloc[num, 4:] = categories_sorted.index.values[0:10]
    
barce_venues_sorted.head()

,Neighborhood,Cluster,Latitude,Longitude,No.1,No.2,No.3,No.4,No.5,No.6,No.7,No.8,No.9,No.10
0,Can Baró,4,41.416966,2.162693,Bar,Spanish Restaurant,Park,Gym,Bakery,Grocery Store,Café,Plaza,Tapas Restaurant,Restaurant
1,Dreta de l'Eixample,3,41.394124,2.166471,Hotel,Tapas Restaurant,Boutique,Mediterranean Restaurant,Restaurant,Spanish Restaurant,Hostel,Bookstore,Clothing Store,Bakery
2,El Baix Guinardó,1,41.411441,2.168510,Restaurant,Tapas Restaurant,Bakery,Plaza,Bar,Italian Restaurant,Vegetarian / Vegan Restaurant,Pizza Place,Brewery,Hotel
3,El Carmel,4,41.425591,2.154959,Spanish Restaurant,Tapas Restaurant,Restaurant,Plaza,Bakery,Park,Mediterranean Restaurant,Metro Station,Pizza Place,Café
4,El Guinardó,1,41.417734,2.173632,Restaurant,Tapas Restaurant,Bakery,Spanish Restaurant,Italian Restaurant,Café,Bar,Brewery,Hotel,Gym


Next, distinguish the condition of neighborhoods in each cluster to find out their similarities.

In [42]:
print("Cluster 1")
barce_venues_sorted.loc[barce_venues_sorted['Cluster']==0,:].iloc[:,[0]+list(range(4, 14))]

Cluster 1


,Neighborhood,No.1,No.2,No.3,No.4,No.5,No.6,No.7,No.8,No.9,No.10
10,La Vall d'Hebron.,Park,Spanish Restaurant,Grocery Store,Bakery,Tennis Court,Metro Station,Tapas Restaurant,Plaza,Argentinian Restaurant,Chinese Restaurant
11,Montbau,Spanish Restaurant,Grocery Store,Restaurant,Plaza,Park,Soccer Field,Soccer Stadium,Hotel,Tennis Court,Chinese Restaurant
15,Sant Genís dels Agudells,Plaza,Grocery Store,Spanish Restaurant,Park,Soccer Stadium,Tapas Restaurant,Tennis Court,Argentinian Restaurant,Outdoor Sculpture,Café
28,la Teixonera,Park,Grocery Store,Spanish Restaurant,Plaza,Scenic Lookout,Tapas Restaurant,Tennis Court,Hotel,Chinese Restaurant,Mountain


In [43]:
print("Cluster 2")
barce_venues_sorted.loc[barce_venues_sorted['Cluster']==1,:].iloc[:,[0]+list(range(4, 14))]

Cluster 2


,Neighborhood,No.1,No.2,No.3,No.4,No.5,No.6,No.7,No.8,No.9,No.10
2,El Baix Guinardó,Restaurant,Tapas Restaurant,Bakery,Plaza,Bar,Italian Restaurant,Vegetarian / Vegan Restaurant,Pizza Place,Brewery,Hotel
4,El Guinardó,Restaurant,Tapas Restaurant,Bakery,Spanish Restaurant,Italian Restaurant,Café,Bar,Brewery,Hotel,Gym
5,El Putget i Farró,Bakery,Pizza Place,Plaza,Italian Restaurant,Wine Bar,Mediterranean Restaurant,Park,Japanese Restaurant,Café,Deli / Bodega
13,Sagrada Família,Italian Restaurant,Burger Joint,Hotel,Restaurant,Bar,Plaza,Pizza Place,Mexican Restaurant,Steakhouse,Sandwich Place
16,Sant Gervasi - Galvany,Restaurant,Spanish Restaurant,Mediterranean Restaurant,Pizza Place,Tapas Restaurant,Japanese Restaurant,Italian Restaurant,Deli / Bodega,French Restaurant,Wine Bar
18,Sarrià,Hotel,Mediterranean Restaurant,Bakery,Grocery Store,Café,Burger Joint,Japanese Restaurant,Park,Gym / Fitness Center,Restaurant
22,Vila de Gràcia,Tapas Restaurant,Plaza,Restaurant,Café,Cocktail Bar,Mediterranean Restaurant,Bar,Pizza Place,Wine Bar,Indie Movie Theater
23,el Camp d'en Grassot i Gràcia Nova,Tapas Restaurant,Plaza,Mediterranean Restaurant,Indie Movie Theater,Vegetarian / Vegan Restaurant,Restaurant,Bar,Bakery,Hotel,Japanese Restaurant
26,la Maternitat i Sant Ramon,Hotel,Pizza Place,Café,Spanish Restaurant,Bar,Bakery,Nightclub,Park,Coffee Shop,Mediterranean Restaurant
29,les Corts,Restaurant,Pizza Place,Sandwich Place,Café,Spanish Restaurant,Hotel,Burger Joint,Supermarket,Cocktail Bar,Asian Restaurant


In [44]:
print("Cluster 3")
barce_venues_sorted.loc[barce_venues_sorted['Cluster']==2,:].iloc[:,[0]+list(range(4, 14))]

Cluster 3


,Neighborhood,No.1,No.2,No.3,No.4,No.5,No.6,No.7,No.8,No.9,No.10
19,Tibidabo i les Planes,Building,BBQ Joint,Soccer Field,Fountain,National Park,General Entertainment,Scenic Lookout,Light Rail Station,Farmers Market,Tourist Information Center
21,Vallvidrera,Light Rail Station,Building,BBQ Joint,Soccer Field,Fountain,National Park,Scenic Lookout,Lake,Farmers Market,Restaurant


In [45]:
print("Cluster 4")
barce_venues_sorted.loc[barce_venues_sorted['Cluster']==3,:].iloc[:,[0]+list(range(4, 14))]

Cluster 4


,Neighborhood,No.1,No.2,No.3,No.4,No.5,No.6,No.7,No.8,No.9,No.10
1,Dreta de l'Eixample,Hotel,Tapas Restaurant,Boutique,Mediterranean Restaurant,Restaurant,Spanish Restaurant,Hostel,Bookstore,Clothing Store,Bakery
6,Fort Pienc,Hotel,Coffee Shop,Mediterranean Restaurant,Bistro,Hostel,Burger Joint,Spanish Restaurant,Restaurant,Tapas Restaurant,Breakfast Spot
8,L'Antiga Esquerra de l'Eixample,Hotel,Spanish Restaurant,Mediterranean Restaurant,Cocktail Bar,Tapas Restaurant,Japanese Restaurant,Beer Bar,Wine Bar,Pizza Place,Restaurant
9,La Nova Esquerra de l'Eixample,Hotel,Spanish Restaurant,Coffee Shop,Tapas Restaurant,Mediterranean Restaurant,Café,Japanese Restaurant,Korean Restaurant,Gym,Restaurant
14,Sant Antoni,Cocktail Bar,Tapas Restaurant,Coffee Shop,Mediterranean Restaurant,Hotel,Bar,Beer Bar,Japanese Restaurant,Spanish Restaurant,Wine Bar


In [46]:
print("Cluster 5")
barce_venues_sorted.loc[barce_venues_sorted['Cluster']==4,:].iloc[:,[0]+list(range(4, 14))]

Cluster 5


,Neighborhood,No.1,No.2,No.3,No.4,No.5,No.6,No.7,No.8,No.9,No.10
0,Can Baró,Bar,Spanish Restaurant,Park,Gym,Bakery,Grocery Store,Café,Plaza,Tapas Restaurant,Restaurant
3,El Carmel,Spanish Restaurant,Tapas Restaurant,Restaurant,Plaza,Bakery,Park,Mediterranean Restaurant,Metro Station,Pizza Place,Café
7,Horta,Spanish Restaurant,Tapas Restaurant,Grocery Store,Restaurant,Plaza,Mediterranean Restaurant,Park,Hostel,Hotel,Food & Drink Shop
12,Pedralbes,Hotel,Garden,Gym / Fitness Center,Mediterranean Restaurant,Park,Restaurant,Café,Bar,College Cafeteria,Spanish Restaurant
17,Sant Gervasi - la Bonanova,Bakery,Mediterranean Restaurant,Spanish Restaurant,Seafood Restaurant,Plaza,Restaurant,Café,Grocery Store,Bar,Park
20,Vallcarca i els Penitents.,Park,Plaza,Café,Bakery,Hotel,Grocery Store,Supermarket,Spanish Restaurant,Breakfast Spot,Mediterranean Restaurant
24,la Clota,Spanish Restaurant,Plaza,Supermarket,Park,Restaurant,Metro Station,Soccer Field,Café,Sandwich Place,Food & Drink Shop
25,la Font d'en Fargues,Tapas Restaurant,Spanish Restaurant,Restaurant,Bar,Café,Grocery Store,Plaza,Supermarket,Italian Restaurant,Pizza Place
27,la Salut,Plaza,Park,Restaurant,Bar,Hotel,Tapas Restaurant,Café,Mediterranean Restaurant,Italian Restaurant,Scenic Lookout


According to the characteristics of each cluster, it is easy to figure out cluster 1 and 3 are holiday resorts. Besides, cluster 4 is too far away from ESADE Business School. Cluter 2 and 5 can be perfect final candidates, they include restaurants, cafés, bars, plaza, groceries and parks. All make them suitable neighborhoods for me to live. 

In [135]:
barce_tolive = barce_venues_means.loc[(barce_venues_sorted['Cluster']==1) | (barce_venues_sorted['Cluster']==4)]
barce_tolive

,Neighborhood,Cluster,Accessories Store,African Restaurant,American Restaurant,Animal Shelter,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bistro,Board Shop,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Butcher,Cafeteria,Café,Cambodian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doctor's Office,Dog Run,Donut Shop,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Multiplex,Museum,Music Venue,National Park,Nightclub,Noodle House,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pharmacy,Photography Lab,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Restaurant,Road,Rock Club,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio,District,Latitude,Longitude
0,Can Baró,4,0.00,0.00,0.000000,0.00,0.0,0.000000,0.00,0.010000,0.00,0.000000,0.010000,0.00,0.050000,0.070000,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.020000,0.000000,0.00,0.0,0.010000,0.00,0.00,0.040000,0.01,0.00,0.030000,0.0,0.000000,0.0,0.000000,0.000000,0.010000,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.000000,0.00,0.02,0.00,0.010000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.010000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.000000,0.000000,0.0,0.010000,0.0,0.0,0.00,0.000000,0.01,0.0,0.00,0.040000,0.050000,0.000000,0.0,0.0,0.00,0.00,0.0,0.020000,0.00,0.00,0.000000,0.000000,0.010000,0.00,0.000000,

Pick out top 20 common venues in cluster 2 and 5.

In [78]:
barce_tolive.mean().sort_values().to_frame().sort_values(by=0, ascending=False).iloc[3:23,:]

,0
Restaurant,0.051725
Spanish Restaurant,0.046304
Tapas Restaurant,0.043235
Plaza,0.041651
Mediterranean Restaurant,0.038035
Café,0.037032
Bakery,0.036278
Hotel,0.033610
Park,0.033396
Pizza Place,0.031454


Only keep the means of venues as the matrix.

In [105]:
barce_tolive_matrix = barce_tolive.drop(columns=['Neighborhood', 'Cluster', 'District', 'Latitude', 'Longitude'])
barce_tolive_matrix

,Accessories Store,African Restaurant,American Restaurant,Animal Shelter,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bistro,Board Shop,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Butcher,Cafeteria,Café,Cambodian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doctor's Office,Dog Run,Donut Shop,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Multiplex,Museum,Music Venue,National Park,Nightclub,Noodle House,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pharmacy,Photography Lab,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Restaurant,Road,Rock Club,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio
0,0.00,0.00,0.000000,0.00,0.0,0.000000,0.00,0.010000,0.00,0.000000,0.010000,0.00,0.050000,0.070000,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.020000,0.000000,0.00,0.0,0.010000,0.00,0.00,0.040000,0.01,0.00,0.030000,0.0,0.000000,0.0,0.000000,0.000000,0.010000,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.000000,0.00,0.02,0.00,0.010000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.010000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.000000,0.000000,0.0,0.010000,0.0,0.0,0.00,0.000000,0.01,0.0,0.00,0.040000,0.050000,0.000000,0.0,0.0,0.00,0.00,0.0,0.020000,0.00,0.00,0.000000,0.000000,0.010000,0.00,0.000000,0.0,0.000000,0.010000,0.00,0.000000,0.00,0.00,0.00,0.020000,

Next, I will create a preference vector of myself, by asigning a score range from 0 to 10 to each venue. 

In [114]:
pre_dict = {
    'Art Museum': 5,
    'Bakery': 6,
    'Bar': 3,
    'Cafeteria': 3,
    'Café': 3,
    'Chinese Restaurant': 8,
    'Convenience Store': 6,
    'Fast Food Restaurant': 6,
    'Food & Drink Shop': 6,
    'Football Stadium': 3,
    'Fruit & Vegetable Store': 8,
    'Grocery Store': 8,
    'Japanese Restaurant': 4,
    'Market': 5,
    'Mediterranean Restaurant': 3,
    'Museum': 5,
    'Restaurant': 4,
    'Soccer Field': 3,
    'Soccer Stadium': 3,
    'Spanish Restaurant': 4,
    'Supermarket': 5,
    'Tapas Restaurant': 4
}


my_score = barce_tolive_matrix.drop(barce_tolive_matrix.index)
my_score = my_score.append(pre_dict, ignore_index=True)
my_score = my_score.fillna(0)
my_score

,Accessories Store,African Restaurant,American Restaurant,Animal Shelter,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bistro,Board Shop,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Butcher,Cafeteria,Café,Cambodian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doctor's Office,Dog Run,Donut Shop,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Multiplex,Museum,Music Venue,National Park,Nightclub,Noodle House,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pharmacy,Photography Lab,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Restaurant,Road,Rock Club,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,6.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,6.0,0.0,0.0,3.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

Then, multiply the neighborhoods matrix by score vector, I get an array of final preference scores.

In [116]:
barce_tolive_matrix_vector = barce_tolive_matrix.values
my_score_vector = my_score.values.T
score_array = np.dot(barce_tolive_matrix_vector, my_score_vector)
score_array

array([[2.15      ],
       [1.36      ],
       [2.01219512],
       [1.8       ],
       [1.51      ],
       [2.29113924],
       [1.09090909],
       [1.05      ],
       [1.56      ],
       [2.13      ],
       [1.98      ],
       [1.72      ],
       [1.37      ],
       [1.34      ],
       [1.79245283],
       [2.18604651],
       [1.63      ],
       [1.30263158],
       [1.22      ],
       [1.37      ]])

It is quite simple to choose the top 5 suitable neighborhoods and place them on the map.

In [129]:
pd.Series(score_array.T[0])

0     2.150000
1     1.360000
2     2.012195
3     1.800000
4     1.510000
5     2.291139
6     1.090909
7     1.050000
8     1.560000
9     2.130000
10    1.980000
11    1.720000
12    1.370000
13    1.340000
14    1.792453
15    2.186047
16    1.630000
17    1.302632
18    1.220000
19    1.370000
dtype: float64

In [136]:
barce_tolive.reset_index(drop=True, inplace=True)
barce_tolive.insert(1, 'Score', pd.Series(score_array.T[0]))
barce_tolive

,Neighborhood,Score,Cluster,Accessories Store,African Restaurant,American Restaurant,Animal Shelter,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bistro,Board Shop,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Butcher,Cafeteria,Café,Cambodian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doctor's Office,Dog Run,Donut Shop,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Multiplex,Museum,Music Venue,National Park,Nightclub,Noodle House,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pharmacy,Photography Lab,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Restaurant,Road,Rock Club,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio,District,Latitude,Longitude
0,Can Baró,2.150000,4,0.00,0.00,0.000000,0.00,0.0,0.000000,0.00,0.010000,0.00,0.000000,0.010000,0.00,0.050000,0.070000,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.020000,0.000000,0.00,0.0,0.010000,0.00,0.00,0.040000,0.01,0.00,0.030000,0.0,0.000000,0.0,0.000000,0.000000,0.010000,0.000000,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.000000,0.00,0.02,0.00,0.010000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.010000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.000000,0.000000,0.0,0.010000,0.0,0.0,0.00,0.000000,0.01,0.0,0.00,0.040000,0.050000,0.000000,0.0,0.0,0.00,0.00,0.0,0.020000,0.00,0.00,0.000000,0.000000,0.010000

In [143]:
barce_final = barce_tolive.iloc[:,[0,1,-2,-1]].sort_values(by='Score', ascending=False).head(5)
barce_final.reset_index(drop=True, inplace=True)
barce_final

,Neighborhood,Score,Latitude,Longitude
0,Horta,2.291139,41.433021,2.159088
1,la Font d'en Fargues,2.186047,41.424364,2.165320
2,Can Baró,2.150000,41.416966,2.162693
3,Sant Gervasi - la Bonanova,2.130000,41.409637,2.129637
4,El Carmel,2.012195,41.425591,2.154959


In [146]:
final_map = folium.Map(location=[barce_final.mean()['Latitude'], barce_final.mean()['Longitude']], zoom_start=13)

markers_colors = []
for lat, lon, neigh in zip(barce_final['Latitude'], barce_final['Longitude'], barce_final['Neighborhood']):
    label = folium.Popup(str(neigh), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=15,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.5).add_to(final_map)
    
folium.Marker([esade_lat, esade_lng], popup='ESADE').add_to(final_map)
       
final_map

Finally! I find top 5 neighborhoods for me to book apartment in autumn semester. They are ***Horta, la Font d'en Fargues, Can Baró, Sant Gervasi - la Bonanova, and El Carmel.***